<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Negative-Sampling" data-toc-modified-id="Negative-Sampling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Negative Sampling</a></span></li></ul></div>

In [484]:
# for col, z in inputs.items():
#     print(col, z)
#     embedding_mlr_list[col] = Embedding(input_dim=data[col].nunique(), output_dim=1, name=f"Embedding_{col}")(z)

# embedding_mlr_list = []
# for col in data.columns:
#     if col != "cnt":
#         embedding_mlr_list.append(Embedding(input_dim= data[col].nunique(), output_dim=1, name=f"{col}_lr_emb")(inputs[col]))

# class LR(Layer):
#     def build(self, input_shape):
#         # 레이어 연결시 0으로 초기화된 학습 가중치 생성
#         self.b = self.add_weight(shape=(1, ), initializer="zeros", trainable=True)
#         super().build(input_shape)
    
#     def call(self, inputs, **kwargs):
#         logits = tf.add_n(inputs) + self.b
#         return logits

# # Order-2 Part
# dim = 8
# embedding_fm_list = []
# for col in data.columns:
#     if col != "cnt":
# #         fm_embedding_mlr_list.append(Embedding(input_dim= data[col].nunique(), output_dim=dim, name=f"{col}_fm_emb")(inputs[col]))
#         print(Embedding(input_dim= data[col].nunique(), output_dim=dim, name=f"{col}_fm_emb").get_weights)

# batch_size = 512

# train_set = tf.data.Dataset.from_tensor_slices(({k:v.values.astype("int16") for k, v in train_x.iteritems()}, train_y.values)).shuffle(len(train_x)).batch(batch_size, drop_remainder=True)
# test_set = tf.data.Dataset.from_tensor_slices(({k:v.values.astype("int16") for k, v in test_x.iteritems()}, test_y.values)).shuffle(len(test_x)).batch(batch_size, drop_remainder=True)

# cat2idx = {}
# cat2size = {}
# for col in data.columns:
#     if col != "cnt":
#         data[col] = data[col].astype("category")
        
#         cat2idx[col] = {cat:idx for idx, cat in enumerate(data[col].cat.categories)}
#         cat2size[col] = len(data[col].cat.categories)

# for col, dic in cat2idx.items():
#     data[col] = data[col].map(dic)

In [485]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn
from sklearn.model_selection import train_test_split
import random
from tqdm.notebook import tqdm

plt.style.use("dark_background")

In [486]:
data = pd.read_csv("./Datasets/Frappe/frappe.csv")
metadata = pd.read_csv("./Datasets/Frappe/frappe_meta.csv", sep="\t")

# "특정 상황에서 고객이 한번 쯤 사용해볼 것 같은 어플을 찾는 추천시스템"으로 목표를 설정해 보겠습니다. 즉, 특정 상황에서 유저가 한번도 사용하지 않았던 어플 중 가장 관심있어 할만한 어플을 찾아주는 문제로 정의할 수 있습니다.
# 1번 이상 로그가 있는 Context에 대해서는 모두 "클릭을 했다" 라는 의미로 `True`로 변경해줍니다.
data["cnt"] = True

# Modeling

In [487]:
output_dim = 8
inputs = dict()
embedding_mlr_list = list()
embedding_fm_list = list()
for col in data.columns:
    if col != "cnt":
        input_dim = data[col].nunique()
        z = Input(shape=(), name=f"Input_{col}")
        inputs[col] = z

        # Order-1 Part
        z1 = Embedding(input_dim=input_dim, output_dim=1, name=f"Embedding_mlr_{col}")(z)
        embedding_mlr_list.append(z1)
        
        # Order-2 Part
        z2 = Embedding(input_dim=input_dim, output_dim=output_dim, name=f"Embedding_fm_{col}")(z)
        embedding_fm_list.append(z2)
logits_mlr = tf.Variable((0.,)) + tf.math.add_n(embedding_mlr_list)

# Order-2 Part
z = tf.stack(embedding_fm_list, axis=1)
logits_fm = 1/2*tf.math.reduce_sum(tf.math.square(tf.math.reduce_sum(z, axis=1)) - tf.math.reduce_sum(tf.math.square(z), axis=1), axis=1, keepdims=True)

# Order-3 Part
z = Concatenate(axis=1)(embedding_fm_list)
z = Dense(units=256, activation="relu")(z)
z = Dropout(rate=0.5)(z)
z = Dense(units=128, activation="relu")(z)
z = Dropout(rate=0.5)(z)
z = Dense(units=64, activation="relu")(z)
z = Dropout(rate=0.5)(z)
logits_mlp = Dense(units=1)(z)

z = Add()([logits_mlr, logits_fm, logits_mlp])
logits = sigmoid(z)
model = Model(inputs=inputs, outputs=logits, name="deepfm")

model.compile(optimizer=Adagrad(), loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "deepfm"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_user (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 Input_item (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 Input_daytime (InputLayer)     [(None,)]            0           []                               
                                                                                                  
 Input_weekday (InputLayer)     [(None,)]            0           []                               
                                                                                             

# Negative Sampling
- 현재 해당 데이터는 특정 컨택스트에서 고객의 앱에 대한 사용 여부로서 "Positive" 데이터만 존재하고 있습니다. 즉, 고객 행동의 긍정적인 행동만 기록되어있다는 것입니다. 우리는 고객이 클릭하지 않은 앱에 대한 사용 여부, 즉 Positive 데이터 한개당 Negative 데이터 3개씩을 생성하되 item 이외의 다른 feature는 고정하고 item을 무작위로 결정하는 식으로 진행해보도록 하겠습니다.

In [488]:
def negative_sampling(data, n):
    data_neg = pd.concat([data]*n)
    data_neg = data_neg.reset_index(drop=True)
    item_cnt = data.groupby(["item"])["cnt"].sum()
    # data_neg["item"] = random.choices(item_cnt.index, k=len(data_neg), weights=item_cnt.values)
    data_neg["item"] = random.choices(item_cnt.index, k=len(data_neg))
    data_neg["cnt"] = False

    data = pd.concat([data, data_neg], axis=0)
    data = data.drop_duplicates(data.columns.drop(["cnt"]))
    data = data.reset_index(drop=True)
    return data

In [489]:
for col in data.columns:
    if col != "cnt":
        data[col] = pd.Categorical(data[col])
        data[col] = data[col].cat.codes

# tr_val, te = train_test_split(data, test_size=0.1, shuffle=True, stratify=data["cnt"])
tr_val, te = train_test_split(data, test_size=0.1, shuffle=True)

tr_val = negative_sampling(tr_val, 3)
te = negative_sampling(te, 3)

tr_val_X = tr_val.drop(["cnt"], axis=1)
te_X = te.drop(["cnt"], axis=1)

tr_val_X = {col:tr_val_X[col].values for col in tr_val_X.columns}
te_X = {col:te_X[col].values for col in te_X.columns}

tr_val_y = tr_val[["cnt"]].values
te_y = te[["cnt"]].values

es = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=3)
model_path = "./frappe_deepfm.h5"
mc = ModelCheckpoint(filepath=model_path, monito="val_acc", verbose=1, save_best_only=True)
batch_size = 256
hist = model.fit(x=tr_val_X, y=tr_val_y, validation_split=0.2, shuffle=True, batch_size=batch_size, epochs=3, verbose=2, callbacks=[es, mc])

Epoch 1/3

Epoch 00001: val_loss improved from inf to 0.44092, saving model to .\frappe_deepfm.h5
1042/1042 - 7s - loss: 0.6457 - acc: 0.6707 - val_loss: 0.4409 - val_acc: 1.0000 - 7s/epoch - 6ms/step
Epoch 2/3


C:\Users\82104\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


KeyboardInterrupt: 

In [ ]:
tr_val_X

In [ ]:
te_y

In [ ]:
model.predict(te_X)

In [ ]:
fig, axes = plt.subplots(figsize=(8, 8))
axes.plot(hist.history["loss"][1:], label="loss");
axes.plot(hist.history["val_loss"][1:], label="val_loss");
axes.legend();

In [ ]:
model.predict(te_X)

In [ ]:
te_y